# 🚀 Conception Automatique de Notebook par Agents IA

Bienvenue dans ce démonstrateur d'intelligence collective !  
Trois agents spécialisés collaborent pour développer et valider votre notebook :

- **🧑💻 Admin** : élabore et finalise les spécifications dans les cellules Markdown.
- **🤖 Codeur** : implémente le code dans les cellules de type *code*.
- **🔍 Relecteur** : vérifie le travail du codeur et valide (ou non) ses modifications.

## Comment démarrer

1. Configurez votre tâche ci-dessous ⤵  
2. Lancez l'orchestration avec le bouton « Play » ou « Exécuter »  
3. Observez la collaboration en direct : chaque agent intervient à tour de rôle  
4. Les cellules suivantes s’exécuteront automatiquement après validation

**Conseil** : Dans votre explorateur de fichiers, ouvrez le notebook cible (nommé "Notebook-Generated.ipynb" si vous avez choisi une tâche, ou bien le notebook téléversé) pour voir ses cellules évoluer en temps réel !


## Configuration du Projet

Dans cette section, plusieurs modes vous sont proposés pour définir la tâche à réaliser :

- **Aléatoire** : choisit une tâche de manière aléatoire parmi une liste préétablie.  
- **Bibliothèque** : vous sélectionnez la tâche désirée dans un menu déroulant.  
- **Personnalisé** : vous décrivez librement votre tâche.  
- **Importer** : vous téléversez votre propre notebook `.ipynb`.

Cliquez sur **Valider** pour confirmer votre choix. Les cellules suivantes prendront automatiquement en compte cette configuration.


In [ ]:
# %% Configuration du projet - UI
%pip install ipywidgets==8.1.5 jupyter-ui-poll==1.0.0 --quiet

import time
import random

import ipywidgets as widgets
from IPython.display import display
from jupyter_ui_poll import ui_events

# local code
from importlib.machinery import SourceFileLoader
scheduler = SourceFileLoader("nurse_scheduler", "../src/nrp_or-tools.py").load_module()  # use correct solver.py path

# ---- Tâches prédéfinies proposées ----
POSSIBLE_TASKS = [
    "Créer un assistant conversationnel pour la planification infirmière avec CP-SAT"
]

# ---- Widgets de configuration ----
task_selector = widgets.Dropdown(
    options=POSSIBLE_TASKS, 
    description='Tâche :', 
    style={'description_width': 'initial'}
)
custom_task = widgets.Textarea(
    value="Assistant de planification infirmière - Veuillez décrire vos besoins en matière de plannings",
    layout={'width': '90%', 'height': '120px'}
)
uploader    = widgets.FileUpload(accept='.ipynb', multiple=False)
submit_btn  = widgets.Button(description="Valider", button_style='success', icon='rocket')

tabs = widgets.Tab()
tabs.children = [
    widgets.VBox([widgets.HTML("<i>Une tâche aléatoire sera générée</i>")]),
    widgets.VBox([widgets.Label("Choisissez une tâche type :"), task_selector]),
    widgets.VBox([widgets.Label("Écrivez vos instructions :"), custom_task]),
    widgets.VBox([widgets.Label("Uploader votre notebook :"), uploader])
]
tabs.set_title(0, '🎲 Aléatoire')
tabs.set_title(1, '📚 Bibliothèque')
tabs.set_title(2, '✨ Personnalisé')
tabs.set_title(3, '📤 Importer')

# ---- Stockage de la configuration ----
config = {
    'mode': None,
    'task_description': None,
    'uploaded_file': None
}

# Flag qui indique si la config est prête
config_ready = False

def on_submit(_):
    """Callback déclenché au clic du bouton."""
    global config_ready
    try:
        # Récupérer la sélection d'onglet pour choisir le mode
        config['mode'] = tabs.selected_index
        
        # Si mode = "Uploader" (onglet 3)
        if config['mode'] == 3:
            if uploader.value:
                config['uploaded_file'] = uploader.value[0]
        else:
            # Sinon on choisit la tâche
            if config['mode'] == 0:
                config['task_description'] = random.choice(POSSIBLE_TASKS)
            elif config['mode'] == 1:
                config['task_description'] = task_selector.value
            elif config['mode'] == 2:
                config['task_description'] = custom_task.value

        submit_btn.disabled = True
        print("Configuration validée !")
        
    except Exception as e:
        print(f"Erreur pendant la configuration : {e}")

    # On met le flag True pour sortir de la boucle
    config_ready = True

submit_btn.on_click(on_submit)

# ---- Affichage ----
display(widgets.HTML("<h3>🔧 Configuration du Projet</h3>"))
display(tabs)
display(submit_btn)

# ---- Boucle bloquante synchrone ----
print("En attente du clic sur Valider...")

with ui_events() as poll:
    while not config_ready:
        poll(10)
        time.sleep(0.1)

print("✅ Config terminée, vous pouvez exécuter les cellules suivantes !")



Note: you may need to restart the kernel to use updated packages.


HTML(value='<h3>🔧 Configuration du Projet</h3>')

Button(button_style='success', description='Valider', icon='rocket', style=ButtonStyle())

En attente du clic sur Valider...


KeyboardInterrupt: 

In [ ]:
# %% Conversation Handler
from semantic_kernel import Kernel, ContextVariables
from semantic_kernel.planning import ActionPlanner
from semantic_kernel.core_plugins import TimePlugin
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion

class SchedulingParameters:
    def __init__(self):
        self.num_nurses = None
        self.num_days = None  
        self.shifts = ['M', 'A', 'N']
        self.max_consecutive_days = 5
        self.min_rest_days = 2
        self.preferences = {}

async def collect_scheduling_parameters(kernel: Kernel) -> SchedulingParameters:
    params = SchedulingParameters()
    planner = ActionPlanner(kernel)
    
    # Conversation flow
    questions = [
        ("Combien d'infirmier(ère)s doivent être planifié(e)s ?", "num_nurses", int),
        ("Sur combien de jours ?", "num_days", int),
        ("Nombre maximum de jours consécutifs travaillés ?", "max_consecutive_days", int),
        ("Nombre minimum de jours de repos par semaine ?", "min_rest_days", int)
    ]
    
    for question, param_name, param_type in questions:
        while True:
            try:
                answer = await planner.create_plan_async(f"Demander à l'utilisateur: {question}")
                result = await answer.invoke_async()
                setattr(params, param_name, param_type(result.result))
                break
            except Exception as e:
                print(f"Entrée invalide: {str(e)}. Veuillez réessayer.")
    
    return params

## Configuration du LLM (.env)

Dans cette section, nous allons :

1. Vérifier si un fichier `.env` est présent (et déjà configuré) ou non.  
2. Vous proposer une interface pour saisir ou rappeler :  
   - La clé d’API (`OPENAI_API_KEY`),  
   - L’URL d’un endpoint compatible OpenAI (`OPENAI_BASE_URL`),  
   - Le modèle à utiliser (`OPENAI_CHAT_MODEL_ID`).  
3. Mettre à jour ou créer le fichier `.env` une fois la configuration validée.

Les cellules ultérieures se baseront sur ces informations pour orchestrer les agents.


In [ ]:
%pip install python-dotenv requests jupyter-ui-poll --quiet

import os
import time
import requests

from dotenv import load_dotenv
import ipywidgets as widgets
from IPython.display import display
from jupyter_ui_poll import ui_events

# -------------------------------------------------------------------------
# Fonctions utilitaires
# -------------------------------------------------------------------------
def list_models(api_base, api_key):
    """Retourne un dict avec la liste des modèles ou un champ 'error'."""
    url = f"{api_base}/models"
    headers = {
        "Authorization": f"Bearer {api_key}",
        "Content-Type": "application/json"
    }
    try:
        resp = requests.get(url, headers=headers, timeout=20)
        if resp.status_code == 200:
            return resp.json()  # dict, ex: {"data":[...], "object":"list"}
        else:
            return {"error": f"status={resp.status_code}", "text": resp.text}
    except Exception as e:
        return {"error": str(e)}

# -------------------------------------------------------------------------
# Lecture .env
# -------------------------------------------------------------------------
load_dotenv()

sd_fake = "sk-proj-1234567890"
openai_api_key       = os.getenv("OPENAI_API_KEY", sd_fake).strip()
openai_base_url      = os.getenv("OPENAI_BASE_URL", "").strip()
openai_chat_model_id = os.getenv("OPENAI_CHAT_MODEL_ID", "gpt-4o-mini").strip()

already_configured = (
    openai_api_key != sd_fake
    or (openai_base_url not in ["", "https://api.openai.com/v1"])
)

# Flag indiquant quand la config est OK
env_config_ready = False

# -------------------------------------------------------------------------
# Widgets 
# -------------------------------------------------------------------------
message_output = widgets.Output()

api_key_input = widgets.Text(
    value=openai_api_key if already_configured else "",
    placeholder=f"Ex: {sd_fake}",
    description="Clé API :",
    layout={'width': '80%'}
)

base_url_input = widgets.Text(
    value=openai_base_url if already_configured else "https://api.openai.com/v1",
    placeholder="ex: https://api.my-llm.com/v1",
    description="Endpoint :",
    layout={'width': '80%'}
)

model_dropdown = widgets.Dropdown(
    options=[],  # Vide initialement
    description="Modèle :",
    layout={'width': '80%', 'display': 'none'}  # masqué tant qu'on n'a pas listé
)

list_models_btn = widgets.Button(
    description="Lister modèles",
    button_style='info',
    icon='search'
)

validate_llm_btn = widgets.Button(
    description="Enregistrer .env",
    button_style='success',
    icon='save'
)

ui_box = widgets.VBox([
    api_key_input,
    base_url_input,
    model_dropdown,
    widgets.HBox([list_models_btn, validate_llm_btn])
])

# -------------------------------------------------------------------------
# Callbacks
# -------------------------------------------------------------------------
def on_list_models_click(_):
    """Appelé au clic sur 'Lister modèles'."""
    new_base_url = base_url_input.value.strip()
    new_api_key  = api_key_input.value.strip()
    with message_output:
        message_output.clear_output()
        if not new_base_url or not new_api_key:
            print("⚠️ Veuillez saisir un Endpoint et une clé API avant de lister les modèles.")
            return

        info = list_models(new_base_url, new_api_key)
        if "error" in info:
            print(f"Erreur /models: {info['error']} - {info.get('text','')}")
        else:
            data_list = info.get("data", [])
            if not data_list:
                print("Aucun modèle n'a été retourné (data=[]).")
            else:
                model_ids = [m.get("id", "(inconnu)") for m in data_list]
                model_dropdown.options = model_ids

                # Si le .env mentionne déjà un modèle existant, on le sélectionne
                if openai_chat_model_id in model_ids:
                    model_dropdown.value = openai_chat_model_id
                else:
                    model_dropdown.value = model_ids[0]

                model_dropdown.layout.display = 'block'
                print(f"✅ {len(model_ids)} modèle(s) trouvé(s).")

def on_validate_llm_click(_):
    """Appelé au clic sur 'Enregistrer .env'."""
    global env_config_ready
    new_api_key  = api_key_input.value.strip() or "sk-fake"
    new_base_url = base_url_input.value.strip()

    chosen_model = "gpt-3.5-turbo"
    if model_dropdown.options and (model_dropdown.layout.display != 'none'):
        chosen_model = model_dropdown.value.strip()

    with message_output:
        message_output.clear_output()
        try:
            with open('.env', 'w', encoding='utf-8') as f:
                f.write(f"OPENAI_API_KEY={new_api_key}\n")
                f.write(f"OPENAI_BASE_URL={new_base_url}\n")
                f.write(f"OPENAI_CHAT_MODEL_ID={chosen_model}\n")

            print("✅ Fichier .env créé/mis à jour avec :")
            print(f"   - OPENAI_API_KEY       = {new_api_key}")
            print(f"   - OPENAI_BASE_URL      = {new_base_url or '(API OpenAI officiel)'}")
            print(f"   - OPENAI_CHAT_MODEL_ID = {chosen_model}")

            env_config_ready = True

        except Exception as e:
            print(f"❌ Erreur lors de l'écriture du fichier .env: {str(e)}")

# -------------------------------------------------------------------------
# Suppression des anciens callbacks (si la cellule est rejouée)
# -------------------------------------------------------------------------
list_models_btn._click_handlers.callbacks = []
validate_llm_btn._click_handlers.callbacks = []

list_models_btn.on_click(on_list_models_click)
validate_llm_btn.on_click(on_validate_llm_click)

# -------------------------------------------------------------------------
# Affichage 
# -------------------------------------------------------------------------


# Imprimer un message d'intro
if already_configured:
    print("✅ Configuration LLM détectée dans .env :")
    print(f"   - OPENAI_API_KEY       = {openai_api_key}")
    print(f"   - OPENAI_BASE_URL      = {openai_base_url or '(API officielle)'}")
    print(f"   - OPENAI_CHAT_MODEL_ID = {openai_chat_model_id}")
    print("Aucune saisie supplémentaire n'est requise.")
    env_config_ready = True
else:
    print("Veuillez :\n1) Saisir votre Endpoint et clé API")
    print("2) Cliquer sur [Lister modèles] (pour un endpoint custom)")
    print("3) Cliquer sur [Enregistrer .env] pour finaliser la configuration")
    display(ui_box)
    display(message_output)
    # -------------------------------------------------------------------------
    # Boucle bloquante: attend le clic sur "Enregistrer .env"
    # -------------------------------------------------------------------------
    with ui_events() as poll:
        while not env_config_ready:
            poll(10)
            time.sleep(0.1)

    # Sortie de la boucle => On peut masquer ui_box (facultatif)
    ui_box.layout.display = 'none'

    print("✅ Configuration LLM terminée, vous pouvez exécuter la suite !")


## ▶ Démarrage du Processus

La configuration étant terminée, les étapes suivantes vont se lancer :

1. **Installation des dépendances** : Nous vérifions et installons papermill, nbformat, semantic-kernel, etc.  
2. **Gestion d’état** : nous utilisons la classe `NotebookState` pour piloter le cycle de vie du notebook.  
3. **Plugins** : chaque agent aura un *plugin* lui permettant de lire ou modifier le contenu du notebook.  
4. **Stratégies d'orchestration** : nous définissons quelles actions lancer en fonction de l’état (`specified`, `implemented`, `tested`, `validated`).  
5. **Agents** : définition et configuration des trois agents (`AdminAgent`, `CoderAgent`, `ReviewerAgent`).  
6. **Conversation multi-agents** : la conversation s’enchaîne jusqu’à validation du notebook (ou dépassement du nombre maximal d’itérations).

**Objectif final** : Obtenir un notebook entièrement fonctionnel et validé.


## 1. Installation des dépendances

Nous installons ici les bibliothèques nécessaires :

- **papermill** : pour exécuter des notebooks et injecter des variables,  
- **nbformat** : pour manipuler la structure interne d’un notebook,  
- **semantic-kernel** : pour orchestrer la collaboration entre plusieurs agents (LLM).


In [ ]:
# Cellule Code : Installation des packages requis
# ------------------------------------------------
# Nous installons ici les packages indispensables au pipeline.
%pip install papermill==2.6.0 nbformat==5.10.4 semantic-kernel==1.22.1 --quiet

print("Installation terminée. Si nécessaire, redémarrez le kernel pour activer les nouveaux packages.")


## 2. Import des bibliothèques et configuration

Nous importons :

- Les bibliothèques standard (os, json, logging, etc.).  
- Les bibliothèques *notebook* (nbformat, papermill).  
- `semantic-kernel` pour la gestion de nos agents conversationnels.  
- Un logger coloré pour améliorer la lisibilité et le suivi de l’exécution.


In [ ]:
# Cellule Code : Imports et configuration du logger
# -------------------------------------------------
import os
import json
import hashlib
import logging
import nbformat
import papermill as pm
import random
from datetime import datetime

# Imports liés à Semantic Kernel
from semantic_kernel import Kernel
from semantic_kernel.agents import ChatCompletionAgent, AgentGroupChat
from semantic_kernel.connectors.ai.open_ai import OpenAIChatCompletion
from semantic_kernel.functions import kernel_function
from semantic_kernel.agents.strategies.selection.selection_strategy import SelectionStrategy
from semantic_kernel.agents.strategies.termination.termination_strategy import TerminationStrategy
from semantic_kernel.connectors.ai.function_choice_behavior import FunctionChoiceBehavior
from semantic_kernel.functions.kernel_arguments import KernelArguments

class ColorFormatter(logging.Formatter):
    """
    Un formatter coloré pour rendre les logs plus lisibles.
    """
    colors = {
        'DEBUG': '\033[94m',
        'INFO': '\033[92m',
        'WARNING': '\033[93m',
        'ERROR': '\033[91m',
        'CRITICAL': '\033[91m\033[1m'
    }
    reset = '\033[0m'

    def format(self, record: logging.LogRecord) -> str:
        msg = super().format(record)
        return f"{self.colors.get(record.levelname, '')}{msg}{self.reset}"

logger = logging.getLogger("Orchestration")
logger.setLevel(logging.DEBUG)  # Peut être paramétré via .env ou variable

if not logger.handlers:
    handler = logging.StreamHandler()
    handler.setLevel(logging.DEBUG)
    formatter = ColorFormatter(
        fmt="%(asctime)s [%(levelname)s] %(name)s - %(message)s",
        datefmt="%H:%M:%S"
    )
    handler.setFormatter(formatter)
    logger.addHandler(handler)

logger.info("Configuration initiale terminée.")


## 3. Classe `NotebookState` : gestion de l’état du notebook

Cette classe regroupe :

- La lecture et l’écriture du fichier `.ipynb`,  
- L’exécution via Papermill pour détecter d’éventuelles erreurs,  
- Les transitions d’états du notebook : `specified`, `implemented`, `tested`, `validated`,  
- Les mises à jour ciblées d’une cellule déterminée (recherche par contenu).

Elle centralise toutes les opérations afin que chaque agent puisse y accéder.


In [ ]:
class NotebookState:
    """
    Gère le statut et le contenu d'un notebook.
    
    Les états possibles sont :
      - 'specified'
      - 'implemented'
      - 'tested'
      - 'validated'

    Attributes:
        notebook_path (str): Chemin du fichier notebook (.ipynb).
        _cached_notebook (nbformat.NotebookNode): Le contenu du notebook stocké en mémoire.
        _status (str): L'état courant du notebook.
        _previous_states (list[str]): Historique simple des états antérieurs.
    """

    VALID_STATES = ["specified", "implemented", "tested", "validated"]

    def __init__(self, notebook_path: str) -> None:
        self.notebook_path = notebook_path
        self._cached_notebook = None
        self._status = "specified"
        self._previous_states = []
        self._load_notebook()

    def _load_notebook(self) -> None:
        """Charge le notebook depuis le chemin spécifié, en utilisant nbformat."""
        if not os.path.exists(self.notebook_path):
            raise FileNotFoundError(f"Notebook introuvable: {self.notebook_path}")

        with open(self.notebook_path, "r", encoding="utf-8") as f:
            self._cached_notebook = nbformat.read(f, as_version=4)

        logger.debug(
            f"[NotebookState] Chargé '{self.notebook_path}' avec {len(self._cached_notebook.cells)} cellules."
        )

    def save_notebook(self, path: str = "") -> None:
        """
        Enregistre le notebook sur disque (par défaut au même chemin).
        """
        if not path:
            path = self.notebook_path
        try:
            with open(path, "w", encoding="utf-8") as f:
                nbformat.write(self._cached_notebook, f)
            logger.info(f"[NotebookState] Notebook sauvegardé sous {path}")
        except Exception as e:
            logger.error(f"[save_notebook] Erreur de sauvegarde: {e}")
            raise

    def get_notebook_json(self) -> str:
        """Retourne une représentation JSON (str) du notebook actuellement en mémoire."""
        return json.dumps(self._cached_notebook, indent=2, ensure_ascii=False)

    def log_notebook_state(self, max_length: int = 10000) -> None:
        """Affiche dans les logs une partie du JSON pour diagnostic (tronquée si trop longue)."""
        notebook_json = self.get_notebook_json()
        snippet = notebook_json[:max_length]
        if len(notebook_json) > max_length:
            snippet += ">>> TRUNCATED <<<"
        logger.debug(f"[NotebookState] Current notebook state (truncated):\n{snippet}")

    def get_status(self) -> str:
        """Renvoie l'état courant du notebook."""
        return self._status

    def set_status(self, new_status: str) -> None:
        """
        Met à jour l'état du notebook et log la transition.
        Ne fait rien si new_status est invalide.
        """
        if new_status not in self.VALID_STATES:
            logger.error(f"[NotebookState] État invalide: {new_status}")
            return
        logger.info(f"[NotebookState] Passage de l'état {self._status} → {new_status}")
        self._previous_states.append(self._status)
        self._status = new_status

    def reset_outputs(self) -> None:
        """
        Efface les sorties de toutes les cellules (execution_count, outputs).
        Utile avant ré-exécution si on veut repartir à zéro.
        """
        for cell in self._cached_notebook["cells"]:
            if "outputs" in cell:
                cell["outputs"] = []
            if "execution_count" in cell:
                cell["execution_count"] = None
        logger.debug("[NotebookState] Sorties réinitialisées dans le notebook.")

    def execute_notebook(self) -> bool:
        """
        Exécute le notebook via Papermill et retourne True si tout se passe bien,
        ou False si une exception survient.
        """
        logger.info(f"[NotebookState] Exécution Papermill sur {self.notebook_path}.")
        self.save_notebook()  # Sauvegarde avant exécution

        success = True
        try:
            pm.execute_notebook(
                input_path=self.notebook_path,
                output_path=self.notebook_path,
                kernel_name="python3",
                progress_bar=False,
                log_output=True
            )
        except Exception as e:
            logger.error(f"[execute_notebook] Erreur lors de l'exécution: {e}")
            success = False
        finally:
            self._load_notebook()  # Recharger, car Papermill a peut-être modifié le contenu
            self.save_notebook()
            logger.info("[NotebookState] Notebook mis à jour après exécution (avec sorties).")

        return success

    def update_cell(self, cell_index: int, new_source: str) -> None:
        """
        Met à jour la source d’une cellule (index) et sauvegarde le notebook.
        """
        old_src = self._cached_notebook.cells[cell_index].source
        self._cached_notebook.cells[cell_index].source = new_source
        logger.info(f"[NotebookState] Mise à jour de la cellule {cell_index}")
        logger.debug(f"[NotebookState] Ancien contenu:\n{old_src}")
        logger.debug(f"[NotebookState] Nouveau contenu:\n{new_source}")
        self.save_notebook()

    def find_cell_indices_by_content(self, content_pattern: str) -> list:
        """Retourne la liste des indices de cellules contenant `content_pattern`."""
        indices = []
        for i, c in enumerate(self._cached_notebook.cells):
            if content_pattern in c.source:
                indices.append(i)
        return indices

    def is_approved(self) -> bool:
        """Renvoie True si l'état du notebook est 'validated'."""
        return self._status == "validated"


## 4. Création (ou chargement) du Notebook cible + test de validité

Voici les étapes automatisées :

1. Nous utilisons un template `Notebook-Template.ipynb`, ou bien le fichier `.ipynb` téléversé,  
2. Nous injectons une description de tâche (si elle est choisie aléatoirement, prédéfinie ou personnalisée),  
3. Nous exécutons le notebook pour vérifier qu’aucune erreur fatale n’apparaît,  
4. Nous validons l’intégrité pour confirmer que tout est correctement initialisé.

Si tout se passe bien, la phase d’orchestration multi-agents peut commencer.


In [ ]:

# Cellule Code : Initialisation du notebook ciblé et injection de la tâche
# -----------------------------------------------------------------------
import shutil
import os
import requests

TEMPLATE_URL = "https://raw.githubusercontent.com/jsboige/CoursIA/refs/heads/main/MyIA.AI.Notebooks/GenAI/SemanticKernel/Notebook-Template.ipynb"
TEMPLATE_FILE = "Notebook-Template.ipynb"

def apply_task_description(notebook_state: NotebookState, task_description: str) -> bool:
    """
    Remplace le placeholder {{TASK_DESCRIPTION}} dans la première cellule Markdown appropriée.
    Retourne True si le placeholder a été trouvé et remplacé, False sinon.
    """
    for idx, cell in enumerate(notebook_state._cached_notebook.cells):
        if "{{TASK_DESCRIPTION}}" in cell.source:
            new_source = cell.source.replace("{{TASK_DESCRIPTION}}", task_description)
            notebook_state.update_cell(idx, new_source)
            return True
    return False

DEST_NOTEBOOK = "Notebook-Generated.ipynb"

# Gestion différenciée selon le mode sélectionné
if config['mode'] == 3:  # Mode upload
    if config['uploaded_file']:
        DEST_NOTEBOOK = config['uploaded_file'].name
        with open(DEST_NOTEBOOK, 'wb') as f:
            f.write(config['uploaded_file'].content)
        logger.info(f"Notebook uploadé : {DEST_NOTEBOOK}")
else:  # Modes template (aléatoire, bibliothèque, personnalisé)
    if not os.path.exists(TEMPLATE_FILE):
        print(f"{TEMPLATE_FILE} introuvable, téléchargement depuis {TEMPLATE_URL}")
        try:
            response = requests.get(TEMPLATE_URL, timeout=10)
            response.raise_for_status()  # Gère les erreurs HTTP
            with open(TEMPLATE_FILE, "wb") as f:
                f.write(response.content)
            print(f"Téléchargement terminé, fichier {TEMPLATE_FILE} créé.")
        except Exception as e:
            print(f"Échec du téléchargement : {e}")
    else:
        print(f"Le fichier {TEMPLATE_FILE} existe déjà, aucune action nécessaire.")
    
    shutil.copy2(TEMPLATE_FILE, DEST_NOTEBOOK)
    logger.info(f"Création depuis le template : {TEMPLATE_FILE}")

# Instanciation du notebook state
notebook_state = NotebookState(DEST_NOTEBOOK)
notebook_state.log_notebook_state()

# Injection de la tâche pour les modes template
if config['mode'] != 3:
    changed = apply_task_description(notebook_state, config['task_description'])
    
    if changed:
        logger.info("Placeholder {{TASK_DESCRIPTION}} remplacé avec succès.")
        logger.info("Exécution du notebook pour validation initiale...")
        notebook_state.execute_notebook()
        logger.info("Notebook ré-exécuté après injection de la tâche.")
        notebook_state.log_notebook_state()
    else:
        logger.warning("Aucun placeholder détecté. Vérifiez la cellule Markdown contenant {{TASK_DESCRIPTION}}.")
else:
    logger.info("Mode upload - Aucune injection de tâche nécessaire")

## 5. Architecture de plugins : extension du NotebookState

Nous définissons des **plugins** pour manipuler `NotebookState` :

- **BaseNotebookPlugin** : expose en lecture le notebook (méthode `get_notebook_content()`).  
- **CoderNotebookPlugin** : permet de modifier des cellules de code et de signaler la fin d’implémentation.  
- **ReviewerNotebookPlugin** : exécute le notebook et décide d’approuver ou de refuser.  
- **AdminNotebookPlugin** : finalise ou rejette le notebook, et peut éditer les cellules Markdown.

Chaque plugin est un ensemble de fonctions décorées (`@kernel_function`), utilisables par les agents via Semantic Kernel.


In [ ]:
# ================================
# Plugins actualisés (avec logs)
# ================================
class SolverIntegrationPlugin:
    @kernel_function(name="run_solver")
    def run_solver(self, params: dict) -> str:
        scheduler = NurseScheduler()
        shifts = scheduler.create_model(params)
        solution = scheduler.solve(shifts)
        return solution or "No solution found"

class BaseNotebookPlugin:
    """
    Plugin de base pour manipuler NotebookState.
    Fournit la méthode get_notebook_content() 
    pour récupérer le notebook en JSON.
    """

    def __init__(self, state: NotebookState) -> None:
        self.state = state
        self._get_content_counter = 0  # Pour logguer tous les 5 appels

    @kernel_function(
        name="get_notebook_content",
        description="Renvoie le notebook (format JSON) révisé actuellement."
    )
    def get_notebook_content(self) -> str:
        """
        Retourne la représentation JSON du notebook.
        Loggue un extrait toutes les 5 demandes pour éviter la surcharge.
        """
        self._get_content_counter += 1
        content = self.state.get_notebook_json()

        if (self._get_content_counter % 5) == 0:
            logger.info(f"[BaseNotebookPlugin] get_notebook_content() (appel n°{self._get_content_counter}) - log complet")
            self.state.log_notebook_state(max_length=10000)
        else:
            snippet = content[:200]
            snippet += "..." if len(content) > 200 else ""
            logger.info(f"[BaseNotebookPlugin] get_notebook_content() (appel n°{self._get_content_counter}) -> {snippet}")

        return content


class NotebookEditingMixin:
    """
    Mixin fournissant la fonction d'édition de cellule (update_cell_anyway).
    Peut être hérité par Coder ou Admin, qui ont tous deux besoin d'éditer.
    """

    def update_cell_anyway(self, pattern: str, new_source: str, cell_type: str = None) -> str:
        """
        Recherche la première cellule contenant 'pattern' (dans le code ou markdown),
        puis remplace son contenu par 'new_source', si l'état le permet.
        """
        indices = []
        for idx, cell in enumerate(self.state._cached_notebook.cells):
            content_match = (pattern in cell.source)
            type_match = (cell_type is None) or (cell.cell_type == cell_type)
            if content_match and type_match:
                indices.append(idx)

        if not indices:
            return_message = f"Aucune cellule ({cell_type or 'tout type'}) ne contient '{pattern}'."
        elif len(indices) > 1:
            return_message = f"Plusieurs cellules ({cell_type or 'tout type'}) contiennent '{pattern}'."
        else:
            old_status = self.state.get_status()
            if old_status == "validated":
                return_message = "Édition impossible (notebook déjà validé)."
            else:
                self.state.update_cell(indices[0], new_source)
                return_message = f"Cellule {cell_type} contenant '{pattern}' mise à jour."

        logger.info(f"[NotebookEditingMixin] update_cell_anyway -> {return_message}")
        return return_message


class CoderNotebookPlugin(BaseNotebookPlugin, NotebookEditingMixin):
    """
    Plugin pour l'agent 'Coder'.
    Hérite de BaseNotebookPlugin (lecture) et NotebookEditingMixin (édition).
    """

    @kernel_function(
        name="update_cell_by_content",
        description="Modifie la première cellule de Code contenant 'content_pattern'."
    )
    def update_cell_by_content(self, content_pattern: str, new_source: str) -> str:
        logger.info("[CoderNotebookPlugin] update_cell_by_content()")
        status = self.state.get_status()

        if status not in ["specified", "implemented"]:
            msg = f"Erreur: état '{status}' => modifications bloquées."
        else:
            msg = self.update_cell_anyway(
                pattern=content_pattern,
                new_source=new_source,
                cell_type="code"
            )
        logger.info(f"[CoderNotebookPlugin] update_cell_by_content -> {msg}")
        return msg

    @kernel_function(
        name="finish_implementation",
        description="Déclare le notebook 'implemented' lorsque le code est prêt."
    )
    def finish_implementation(self) -> str:
        logger.info("[CoderNotebookPlugin] finish_implementation()")
        status = self.state.get_status()

        if status == "specified":
            self.state.set_status("implemented")
            msg = "Le notebook passe à l'état 'implemented'."
        elif status == "implemented":
            msg = "Le notebook est déjà en état 'implemented'."
        else:
            msg = f"Impossible de passer en 'implemented' depuis '{status}'."

        logger.info(f"[CoderNotebookPlugin] finish_implementation -> {msg}")
        return msg


class ReviewerNotebookPlugin(BaseNotebookPlugin):
    """
    Plugin pour l'agent 'Reviewer'. 
    Il ne peut pas éditer le notebook, mais peut l'exécuter et approuver (ou non).
    """

    @kernel_function(
        name="validate_notebook",
        description="Exécute le notebook et approuve ou non (approve=True/False)."
    )
    def validate_notebook(self, approve: bool = True) -> str:
        logger.info(f"[ReviewerNotebookPlugin] validate_notebook(approve={approve})")
        status = self.state.get_status()

        if status != "implemented":
            msg = f"Le reviewer ne peut pas valider, état actuel = '{status}'."
        else:
            success = self.state.execute_notebook()
            if not success:
                self.state.set_status("specified")
                msg = ("Erreur d'exécution dans le notebook (voir logs). "
                       "Retour à l'état 'specified' pour corrections.")
            else:
                if approve:
                    self.state.set_status("tested")
                    msg = "Le reviewer approuve => état 'tested'."
                else:
                    self.state.set_status("specified")
                    msg = "Le reviewer refuse => retour à 'specified'."

        logger.info(f"[ReviewerNotebookPlugin] validate_notebook -> {msg}")
        return msg


class AdminNotebookPlugin(BaseNotebookPlugin, NotebookEditingMixin):
    """
    Plugin pour l'agent 'Admin'.
    Peut lire, éditer et valider ou invalider le notebook.
    """

    @kernel_function(
        name="update_markdown_cell",
        description="Modifie la première cellule MARKDOWN contenant 'content_pattern'."
    )
    def admin_edit_markdown_cell(self, content_pattern: str, new_source: str) -> str:
        logger.info("[AdminNotebookPlugin] admin_edit_markdown_cell()")
        msg = self.update_cell_anyway(
            pattern=content_pattern,
            new_source=new_source,
            cell_type="markdown"
        )
        # Après édition, on repasse l'état à 'specified'.
        self.state.set_status("specified")

        logger.info(f"[AdminNotebookPlugin] admin_edit_markdown_cell -> {msg}")
        return msg + " -> Revert à 'specified'."

    @kernel_function(
        name="approve_notebook",
        description="Validation finale: admin_ok=True => 'validated', sinon 'specified'."
    )
    def approve_notebook(self, admin_ok: bool = True) -> str:
        logger.info(f"[AdminNotebookPlugin] approve_notebook(admin_ok={admin_ok})")
        status = self.state.get_status()

        if status != "tested":
            msg = f"Impossible d'approuver: l'état est '{status}' (attendu: 'tested')."
        else:
            if admin_ok:
                self.state.set_status("validated")
                msg = "Notebook validé => état 'validated'."
            else:
                self.state.set_status("specified")
                msg = "Admin refuse => retour à 'specified'."

        logger.info(f"[AdminNotebookPlugin] approve_notebook -> {msg}")
        return msg


## 6. Stratégies d’orchestration

Deux stratégies principales :

1. **ApprovedBasedTerminationStrategy**  
   - Met fin à la conversation dès que le notebook est validé, ou si un nombre maximal d’itérations est atteint.  

2. **NotebookAwareSelectionStrategy**  
   - Sélectionne l’agent en fonction de l’état courant du notebook :  
     - `specified` ⇒ **CoderAgent**  
     - `implemented` ⇒ **ReviewerAgent**  
     - `tested` ⇒ **AdminAgent**  
     - `validated` ⇒ plus d’agent (arrêt de la conversation)  
   - Tient aussi compte de la première intervention pour laisser l’Admin faire ses modifications initiales.


In [ ]:
from pydantic import PrivateAttr

class ApprovedBasedTerminationStrategy(TerminationStrategy):
    """
    Met fin à la conversation si le notebook est validé ou si 
    le nombre d'itérations maximum est atteint.
    """
    _state: NotebookState = PrivateAttr()
    _max_steps: int = PrivateAttr(default=20)

    def __init__(self, state: NotebookState, max_steps: int = 20):
        super().__init__()
        self._state = state
        self._max_steps = max_steps
        self._current_step = 0

    async def should_agent_terminate(self, agent, history) -> bool:
        self._current_step += 1
        is_approved = self._state.is_approved()
        logger.debug(
            f"[TerminationStrategy] Step={self._current_step}/{self._max_steps}, IsApproved={is_approved}"
        )
        if is_approved:
            logger.info("[TerminationStrategy] Notebook approuvé => arrêt.")
            return True
        if self._current_step >= self._max_steps:
            logger.warning(f"[TerminationStrategy] max_steps={self._max_steps} atteint => arrêt.")
            return True
        return False


class NotebookAwareSelectionStrategy(SelectionStrategy):
    """
    Sélectionne quel agent doit parler en fonction de l'état du notebook.
    """
    def __init__(self, state: NotebookState):
        super().__init__()
        self._state = state
        self._has_first_agent_run = False

    def reset(self) -> None:
        self._has_first_agent_run = False

    async def select_agent(self, agents, history):
        current_status = self._state.get_status()
        logger.debug(
            f"[SelectionStrategy] nb_agents={len(agents)}, statut={current_status}, "
            f"first_run={not self._has_first_agent_run}"
        )

        if current_status == "validated":
            logger.info("[SelectionStrategy] Notebook déjà validé => fin.")
            return None

        # Récupérer les agents
        coder = next((a for a in agents if a.name == "CoderAgent"), None)
        reviewer = next((a for a in agents if a.name == "ReviewerAgent"), None)
        admin = next((a for a in agents if a.name == "AdminAgent"), None)

        # Cas particulier : première intervention => Admin
        if not self._has_first_agent_run and current_status == "specified":
            if admin:
                self._has_first_agent_run = True
                logger.info("Première intervention : AdminAgent (révision du Markdown).")
                return admin

        # Cas normal
        if current_status == "specified" and coder:
            to_return = coder
        elif current_status == "implemented" and reviewer:
            to_return = reviewer
        elif current_status == "tested" and admin:
            to_return = admin
        else:
            logger.warning(
                f"[SelectionStrategy] Aucun agent trouvé pour état='{current_status}' => stop."
            )
            to_return = None

        if to_return and not self._has_first_agent_run:
            self._has_first_agent_run = True

        if to_return:
            logger.info(f"[SelectionStrategy] Agent sélectionné : {to_return.name}")
        return to_return


## 7. Création des 3 agents (Coder, Reviewer, Admin)

- Chaque agent dispose d’un `kernel` indépendant, relié à un plugin dédié,  
- Le service ChatCompletion (OpenAI) ou un endpoint custom est configuré via le `.env`,  
- Nous activons le comportement « Auto » pour le choix et l’appel des fonctions,  
- Les rôles :  
  - **CoderAgent** : implémente les cellules de code,  
  - **ReviewerAgent** : exécute et valide (ou non) après relecture,  
  - **AdminAgent** : valide, invalide, ou réédite les spécifications dans le Markdown.


In [ ]:
# Cellule Code : Création des 3 agents et configuration avec prise en compte du .env
# ---------------------------------------------------------------------------------
from openai import AsyncOpenAI

def create_chat_completion_service(service_id: str = "default"):
    """
    Crée une instance de ChatCompletion (OpenAI) en fonction
    des variables d'environnement et du mode choisi.

    Variables .env possibles :
      - OPENAI_API_KEY      : Clé API à utiliser (par défaut, "sk-fake").
      - OPENAI_CHAT_MODEL_ID: Nom du modèle (ex: "gpt-4", "gpt-4o-mini", etc.).
      - OPENAI_BASE_URL     : URL personnalisée pour un endpoint compatible OpenAI.
    """

    # Selon la valeur de GLOBAL_LLM_SERVICE, soit on utilise la version officielle OpenAI,
    # soit on prend un endpoint personnalisé (LLM local, Ollama, vLLM, LMStudio, etc.)
    if openai_base_url.lower() == "":
        logger.info("Utilisation du service OpenAI officiel.")
        return OpenAIChatCompletion(
            service_id=service_id,
            ai_model_id=openai_chat_model_id,
            api_key=openai_api_key
        )
    else:
        logger.info(f"Utilisation d'un endpoint compatible OpenAI : {openai_base_url}")
        # Client "asynchrone" pour requêter un endpoint local ou distant
        openAIClient = AsyncOpenAI(
            api_key=openai_api_key,
            base_url=openai_base_url
        )
        return OpenAIChatCompletion(
            service_id=service_id,
            ai_model_id=openai_chat_model_id,
            async_client=openAIClient
        )

def create_kernel_for_agent(agent_id: str, plugin_instance) -> Kernel:
    """
    Instancie un Kernel SemanticKernel pour l'agent donné,
    en y ajoutant le plugin associé (Coder, Reviewer ou Admin).
    Utilise create_chat_completion_service() pour choisir le backend OpenAI
    (officiel ou endpoint custom) en fonction du .env.
    """
    k = Kernel()

    # Instancier le service de ChatCompletion en fonction du .env
    chat_service = create_chat_completion_service(service_id="default")

    # Ajouter ce service au kernel
    k.add_service(chat_service)

    # Ajouter le plugin de l'agent (Coder, Reviewer, Admin)
    k.add_plugin(plugin_instance, plugin_name=f"{agent_id}_plugin")

    return k

# Instancier les plugins (basés sur NotebookState)
coder_plugin = CoderNotebookPlugin(notebook_state)
reviewer_plugin = ReviewerNotebookPlugin(notebook_state)
admin_plugin = AdminNotebookPlugin(notebook_state)

# Créer un kernel dédié par agent (en tenant compte du .env)
coder_kernel = create_kernel_for_agent("coder_kernel", coder_plugin)
reviewer_kernel = create_kernel_for_agent("reviewer_kernel", reviewer_plugin)
admin_kernel = create_kernel_for_agent("admin_kernel", admin_plugin)

# Configurer l'auto-appel de fonctions
coder_settings = coder_kernel.get_prompt_execution_settings_from_service_id("default")
coder_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

reviewer_settings = reviewer_kernel.get_prompt_execution_settings_from_service_id("default")
reviewer_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

admin_settings = admin_kernel.get_prompt_execution_settings_from_service_id("default")
admin_settings.function_choice_behavior = FunctionChoiceBehavior.Auto()

# Définition des 3 agents conversationnels
coder_agent = ChatCompletionAgent(
    kernel=coder_kernel,
    name="CoderAgent",
    instructions="""Implémenter en utilisant le solver externe:
    1. Importer NurseScheduler depuis nurse_rostering
    2. Initialiser le solver avec les paramètres utilisateur
    3. Appeler create_model() et solve()
    4. Afficher les résultats dans une cellule Markdown"""
)

reviewer_agent = ChatCompletionAgent(
    kernel=reviewer_kernel,
    name="ReviewerAgent",
    instructions=(
        "Vous êtes le **Reviewer**. Votre rôle: vérifier avec le plus grand soin le travail du codeur, l'aiguiller pour d'éventuelles corrections, et valider son travail quand il est parfait.\n"
        "1) Commencez toujours par consulter le notebook complet (get_notebook_content()) après chaque mise à jour.\n"
        "2) Appelez validate_notebook(approve=True/False) selon la qualité.\n"
        "   - True => passage à 'tested'\n"
        "   - False => retour à 'specified'\n"
        "3) Vérifiez scrupuleusement les éventuelles erreurs et la bonne implémentation dans les cellules de code des notions présentées dans les cellules de Markdown.\n"
    ),
    arguments=KernelArguments(settings=reviewer_settings)
)

admin_agent = ChatCompletionAgent(
    kernel=admin_kernel,
    name="AdminAgent",
    instructions="""Vous êtes l'administrateur de planification. Votre rôle:
    1. Collecter tous les paramètres nécessaires via une conversation naturelle
    2. Vérifier la cohérence des contraintes
    3. Générer la spécification Markdown détaillée
    4. Superviser l'exécution du solveur"""
)




# Mettre en place la stratégie de sélection & la stratégie d'arrêt
termination_strategy = ApprovedBasedTerminationStrategy(notebook_state)
selection_strategy = NotebookAwareSelectionStrategy(notebook_state)

# GroupChat pour orchestrer la conversation
group_chat = AgentGroupChat(
    agents=[coder_agent, reviewer_agent, admin_agent],
    selection_strategy=selection_strategy,
    termination_strategy=termination_strategy
)

logger.info("Agents créés et group_chat initialisé avec instructions mises à jour.")


## 8. Boucle de conversation

Nous lançons enfin la conversation multi-agents :

- À chaque itération, l’agent sélectionné dépend de l’état (`specified`, `implemented`, `tested`, `validated`).  
- Les agents peuvent appeler leurs plugins (ex. : `update_cell_by_content`, `validate_notebook`, `approve_notebook`, etc.).  
- La conversation s’arrête dès que le notebook est validé ou si le quota d’itérations est dépassé.

Le statut final du notebook (approuvé ou non) est alors visible dans les logs et dans son contenu.


In [ ]:
import asyncio

async def run_conversation():
    """
    Lance la conversation multi-agents pour finaliser le notebook.
    S'arrête si le notebook est validé ou si on dépasse un max d'itérations.
    """
    try:
        logger.info("Version initiale du notebook :")
        notebook_state.log_notebook_state()

        # Contexte initial pour la conversation
        initial_content = notebook_state.get_notebook_json()
        group_chat.history.add_system_message(f"NOTEBOOK CONTENT:\n{initial_content}")
        group_chat.history.add_user_message("Bonjour, merci de finaliser ce notebook.")

        logger.info("=== Début de la conversation entre agents ===")
        iteration = 0

        async for message in group_chat.invoke():
            iteration += 1
            role = message.name
            content = message.content
            logger.info(f"[STEP {iteration} - {role}] {content}")

            # Vérifier si le notebook est déjà validé
            if notebook_state.is_approved():
                logger.info("Notebook approuvé => fin de la conversation.")
                break

        logger.info("Version finale du notebook, après la conversation :")
        notebook_state.log_notebook_state(max_length=20000)

    except Exception as e:
        logger.error(f"Erreur inattendue: {str(e)}")
    finally:
        logger.info(f"Statut final - Approuvé: {notebook_state.is_approved()}")
        logger.info("=== Fin de la conversation ===")

    # Collect parameters first
    kernel = Kernel()
    kernel.add_plugin(TimePlugin())
    params = await collect_scheduling_parameters(kernel)
    
    # Initialize solver
    solution = kernel.invoke(SolverIntegrationPlugin.run_solver, params)
    notebook_state.update_cell(-1, f"## Solution\n```\n{solution}\n```") 
    
    # Update notebook with results
    results_cell = f"## Résultats de planification\n```\n{solution}\n```"
    notebook_state.update_cell(-1, results_cell)
    notebook_state.set_status("validated")

# Exécuter la conversation
await run_conversation()
